##Team 5 Jupyter notebook - Project 1

In [2]:
#Dependencies
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import numpy as np
import requests
import random
import gmaps
import json
import os

In [4]:
## Create a url API INEGI
url_path = https://www.inegi.org.mx/app/api/indicadores/desarrolladores/jsonxml/INDICATOR/668921,6207061405/es//true/BIE/2.0/1eda5bfc-0e53-ec67-9976-42321b74e950?type=json

SyntaxError: invalid syntax (<ipython-input-4-a8c1ae561545>, line 2)

In [5]:
#Import documentation
df = pd.read_excel("Resources/Centraleselectricas-2.xlsx")
df

,No.,Número de identificación del proyecto asignado por el distribuidor,Nivel de tensión en el punto de interconexión,Número de fases,Capacidad de la central eléctrica (kW),Área Administrativa del Distribuidor (Division),Área Administrativa del Distribuidor (Zona),Estado,Municipio,Fecha Solicitud,Estado en que se encuentra la solicitud: Recibida/Atendida/En lista de espera/Cancelada,Fecha de Interconexión Física,Estudio de interconexión requerido (Si/No),Tecnología,Ubicación por circuito de distribución,Capacidad del circuito de distribución para albergar CE de GD,Ubicación por subestación,Ubicación por Red de Distribución,Ubicación por NodoP,Regímen de contraprestación seleccionado
0,1,62003,BAJA,2,1.65,DIVISION BAJA CALIFORNIA,ZONA TIJUANA,Baja California,Tijuana,2017-05-29,ATENDIDA,2017-05-31,No,FUENTE DE ENERGIA SOLAR,RII04175,No disponible,RIO,No disponible,07RII-69,MEDICIÓN NETA DE ENERGÍA
1,2,64518,BAJA,2,4.88,DIVISION BAJA CALIFORNIA,ZONA TIJUANA,Baja California,Tijuana,2017-04-13,ATENDIDA,2017-04-19,No,FUENTE DE ENERGIA SOLAR,HMO04155,No disponible,HIPODROMO,No disponible,07HMO-69,MEDICIÓN NETA DE ENERGÍA
2,3,63639,BAJA,2,3.72,DIVISION BAJA CALIFORNIA,ZONA TIJUANA,Baja California,Tijuana,2017-05-02,ATENDIDA,2017-05-11,No,FUENTE DE ENERGIA SOLAR,HMO04155,No disponible,HIPODROMO,No disponible,07HMO-69,MEDICIÓN NETA DE ENERGÍA
3,4,63640,BAJA,2,5.58,DIVISION BAJA CALIFORNIA,ZONA TIJUANA,Baja California,Tijuana,2017-04-20,ATENDIDA,2017-04-24,No,FUENTE DE ENERGIA SOLAR,HMO04155,No disponible,HIPODROMO,No disponible,07HMO-69,MEDICIÓN NETA DE ENERGÍA
4,5,66728,BAJA,2,3.72,DIVISION BAJA CALIFORNIA,ZONA TIJUANA,Baja California,Tijuana,2017-04-21,ATENDIDA,2017-04-25,No,FUENTE DE ENERGIA SOLAR,HMO04155,No disponible,HIPODROMO,No disponible,07HMO-69,MEDICIÓN NETA DE ENERGÍA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100332,100333,196291,BAJA,2,1.50,DIVISION VALLE MEXICO SUR ...,ZONA VOLCANES,Estado de México,Amanalco,2019-11-26,EN LISTA DE ESPERA,2019-12-03,NO,FUENTE DE ENERGIA SOLAR ...,IXL53085,2.62,IXTAPALUCA,No Disponible,01IXL-230,MEDICIÓN NETA DE ENERGÍA ...
100333,100334,195138,BAJA,2,2.64,DIVISION VALLE MEXICO SUR ...,ZONA VOLCANES,Estado de México,Calimaya,2019-08-29,EN LISTA DE ESPERA,2019-09-05,NO,FUENTE DE ENERGIA SOLAR ...,JCP53035,1.82,JUCHITEPEC,No Disponible,01JCP-85,MEDICIÓN NETA DE ENERGÍA ...
100334,100335,213873,BAJA,2,1.00,DIVISION VALLE MEXICO SUR ...,ZONA VOLCANES,Estado de México,Tenango del Aire,2019-12-20,EN LISTA DE ESPERA,2019-12-27,NO,FUENTE DE ENERGIA SOLAR ...,JCP53025,2.02,JUCHITEPEC,No Disponible,01JCP-85,MEDICIÓN NETA DE ENERGÍA ...
100335,100336,202369,BAJA,2,9.00,DIVISION VALLE MEXICO SUR ...,ZONA VOLCANES,Estado de México,Tlalmanalco,2019-07-23,EN LISTA DE ESPERA,2019-07-30,NO,FUENTE DE ENERGIA SOLAR ...,AGV53015,3.31,AGUA VIVA,No Disponible,01AGV-85,MEDICIÓN NETA DE ENERGÍA ...


In [6]:
#Clean Data
col=['No.','Número de identificación del proyecto asignado por el distribuidor','Nivel de tensión en el punto de interconexión'
, 'Número de fases','Área Administrativa del Distribuidor (Division)','Área Administrativa del Distribuidor (Zona)', 'Ubicación por circuito de distribución','Capacidad del circuito de distribución para albergar CE de GD',
    'Ubicación por subestación','Ubicación por Red de Distribución','Ubicación por NodoP','Regímen de contraprestación seleccionado','Estudio de interconexión requerido (Si/No)'
    ]
clean_df= df.drop(columns=col)
clean_df['year']=pd.DatetimeIndex(clean_df['Fecha de Interconexión Física']).year
clean_df_loc = clean_df.loc[clean_df["Estado en que se encuentra la solicitud: Recibida/Atendida/En lista de espera/Cancelada"] == "ATENDIDA"]
clean_df_loc = clean_df_loc.drop(columns = ["Estado en que se encuentra la solicitud: Recibida/Atendida/En lista de espera/Cancelada", "Fecha Solicitud", "Fecha de Interconexión Física", "Municipio"])
clean_df_gp = clean_df_loc.groupby(["year", "Estado", "Tecnología"]).sum()
clean_df_gp

Capacidad de la central eléctrica (kW)
year Estado              Tecnología                                                              
2017 Aguascalientes      FUENTE DE ENERGIA SOLAR                                          5322.08
     Baja California     FUENTE DE ENERGIA SOLAR                                          5689.40
     Baja California Sur FUENTE DE ENERGIA SOLAR                                          2853.31
     Campeche            FUENTE DE ENERGIA SOLAR                                           998.20
     Chiapas             FUENTE DE ENERGIA SOLAR                                          1644.83
...                                                                                           ...
2019 Veracruz            FUENTE DE ENERGIA HIDROELECTRICA                                    8.20
                         FUENTE DE ENERGIA SOLAR                                           851.55
     Yucatán             FUENTE DE ENERGIA GAS                                               1.06
                         FUENTE DE ENERGIA SOLAR                                          7376.56
     Zacatecas           FUENTE DE ENERGIA SOLAR                                           265.05

[138 rows x 1 columns]

In [7]:
clean_df_gp.reset_index(inplace = True)
clean_df_gp["Capacidad_kW_2017"] = clean_df_gp.loc[clean_df_gp.year == 2017,"Capacidad de la central eléctrica (kW)"]
clean_df_gp["Capacidad_kW_2018"] = clean_df_gp.loc[clean_df_gp.year == 2018,"Capacidad de la central eléctrica (kW)"]
clean_df_gp["Capacidad_kW_2019"] = clean_df_gp.loc[clean_df_gp.year == 2019,"Capacidad de la central eléctrica (kW)"]
clean_df_gp

,year,Estado,Tecnología,Capacidad de la central eléctrica (kW),Capacidad_kW_2017,Capacidad_kW_2018,Capacidad_kW_2019
0,2017,Aguascalientes,FUENTE DE ENERGIA SOLAR,5322.08,5322.08,NaN,NaN
1,2017,Baja California,FUENTE DE ENERGIA SOLAR,5689.40,5689.40,NaN,NaN
2,2017,Baja California Sur,FUENTE DE ENERGIA SOLAR,2853.31,2853.31,NaN,NaN
3,2017,Campeche,FUENTE DE ENERGIA SOLAR,998.20,998.20,NaN,NaN
4,2017,Chiapas,FUENTE DE ENERGIA SOLAR,1644.83,1644.83,NaN,NaN
...,...,...,...,...,...,...,...
133,2019,Veracruz,FUENTE DE ENERGIA HIDROELECTRICA,8.20,NaN,NaN,8.20
134,2019,Veracruz,FUENTE DE ENERGIA SOLAR,851.55,NaN,NaN,851.55
135,2019,Yucatán,FUENTE DE ENERGIA GAS,1.06,NaN,NaN,1.06
136,2019,Yucatán,FUENTE DE ENERGIA SOLAR,7376.56,NaN,NaN,7376.56


In [8]:
clean_df_gp.loc[clean_df_gp['Capacidad_kW_2017'].isnull(), 'Capacidad_kW_2017'] = 0
clean_df_gp.loc[clean_df_gp['Capacidad_kW_2018'].isnull(), 'Capacidad_kW_2018'] = 0
clean_df_gp.loc[clean_df_gp['Capacidad_kW_2019'].isnull(), 'Capacidad_kW_2019'] = 0
# columns=['level_0','year','Capacidad de la central eléctrica (kW)']
# clean_df_gp= df.drop(columns=col)
clean_df_gp

,year,Estado,Tecnología,Capacidad de la central eléctrica (kW),Capacidad_kW_2017,Capacidad_kW_2018,Capacidad_kW_2019
0,2017,Aguascalientes,FUENTE DE ENERGIA SOLAR,5322.08,5322.08,0.0,0.00
1,2017,Baja California,FUENTE DE ENERGIA SOLAR,5689.40,5689.40,0.0,0.00
2,2017,Baja California Sur,FUENTE DE ENERGIA SOLAR,2853.31,2853.31,0.0,0.00
3,2017,Campeche,FUENTE DE ENERGIA SOLAR,998.20,998.20,0.0,0.00
4,2017,Chiapas,FUENTE DE ENERGIA SOLAR,1644.83,1644.83,0.0,0.00
...,...,...,...,...,...,...,...
133,2019,Veracruz,FUENTE DE ENERGIA HIDROELECTRICA,8.20,0.00,0.0,8.20
134,2019,Veracruz,FUENTE DE ENERGIA SOLAR,851.55,0.00,0.0,851.55
135,2019,Yucatán,FUENTE DE ENERGIA GAS,1.06,0.00,0.0,1.06
136,2019,Yucatán,FUENTE DE ENERGIA SOLAR,7376.56,0.00,0.0,7376.56


In [9]:
#Data cleaning
b = pd.read_excel("Resources/4_itaee_var_anual_retro2.xlsx")
b
clean_state = b[["Estado","Crecimiento ITAEE 2017", "Crecimiento ITAEE 2018", "Crecimiento ITAEE 2019"]]
clean_state

,Estado,Crecimiento ITAEE 2017,Crecimiento ITAEE 2018,Crecimiento ITAEE 2019
0,Aguascalientes,2.459206,3.727672,-0.031717
1,Baja California,3.358687,2.284226,2.052373
2,Baja California Sur,10.708411,16.815057,-7.103514
3,Campeche,-10.446591,-1.422752,-2.153711
4,Coahuila de Zaragoza,5.444967,1.233570,0.171539
5,Colima,3.485108,2.503819,4.320531
6,Chiapas,-3.327992,-2.138943,-2.460577
7,Chihuahua,3.307919,1.956625,2.079865
8,Ciudad de México,3.001314,2.690299,-0.309418
9,Durango,-0.269926,1.451128,0.600691


In [10]:
clean_state.rename(columns = {"Crecimiento ITAEE 2017":"Crecimiento_2017"
                              ,"Crecimiento ITAEE 2018":"Crecimiento_2018"
                               ,"Crecimiento ITAEE 2019":"Crecimiento_2019"},inplace= True)
clean_state


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,Estado,Crecimiento_2017,Crecimiento_2018,Crecimiento_2019
0,Aguascalientes,2.459206,3.727672,-0.031717
1,Baja California,3.358687,2.284226,2.052373
2,Baja California Sur,10.708411,16.815057,-7.103514
3,Campeche,-10.446591,-1.422752,-2.153711
4,Coahuila de Zaragoza,5.444967,1.233570,0.171539
5,Colima,3.485108,2.503819,4.320531
6,Chiapas,-3.327992,-2.138943,-2.460577
7,Chihuahua,3.307919,1.956625,2.079865
8,Ciudad de México,3.001314,2.690299,-0.309418
9,Durango,-0.269926,1.451128,0.600691


In [22]:
solar_irradiation= pd.read_excel("Resources/IrradiaciónSolar.xlsx")
solar_irradiation.head()

,Estado,Estado.1,Ciudad,Ene,Feb,Mar,Abr,May,Jun,Jul,Agos,Sept,Oct,Nov,Dic,Min,Max,Medida
0,Ags,Aguascalientes,Aguascalientes,4.5,5.2,5.9,6.6,7.2,6.3,6.1,5.9,5.7,5.1,4.8,4.0,4.0,7.2,5.6
1,BCS,Baja California Sur,La Paz,4.4,5.5,6.0,6.6,6.5,6.6,6.3,6.2,5.9,5.8,4.9,4.2,4.2,4.2,6.6
2,BC,Baja California,Mexicali,4.1,4.4,5.0,5.6,6.6,7.3,7.0,6.1,6.1,5.5,4.5,3.9,3.9,7.3,5.5
3,BC,Baja California,San Javier,4.2,4.6,5.3,6.2,6.5,7.1,6.4,6.3,6.4,5.1,4.7,3.7,3.7,7.1,5.5
4,BCS,Baja California Sur,San José del Cabo,5.0,5.6,5.8,5.9,6.9,6.1,5.8,6.2,5.8,5.8,5.2,4.4,4.5,6.3,5.7


In [32]:
solar_irradiation_clean=solar_irradiation.drop(columns=["Estado"])
solar_irradiation_clean
solar_irradiation_clean=solar_irradiation_clean.rename(columns={"Estado.1":"Estado"})
solar_irradiation_clean["Estado"]=solar_irradiation_clean["Estado"].str.replace("Yucatan","Yucatán")
solar_irradiation_clean["Estado"]=solar_irradiation_clean["Estado"].str.replace("Nuevo Leon","Nuevo León")
solar_irradiation_clean
solar_irradiation_clean.groupby("Estado").sum()
avg_solar=solar_irradiation_clean.drop(columns=["Ciudad","Ene","Feb","Mar","Abr","May","Jun","Jul","Agos","Sept","Oct","Nov","Dic","Min","Max"])
avg_solar.head()
grouped_df=pd.merge(group_df,avg_solar,how="left",on="Estado")
grouped_df.head()
grouped_df.drop(columns = "year", inplace=True)

grouped_df.drop_duplicates(subset="Estado", inplace=True)
grouped_df



,Estado,Capacidad de la central eléctrica (kW),Capacidad_kW_2017,Capacidad_kW_2018,Capacidad_kW_2019,Crecimiento_2017,Crecimiento_2018,Crecimiento_2019,Medida
0,Aguascalientes,12519.23,5322.08,3963.01,3234.14,12.296030,18.638361,-0.158587,5.6
1,Baja California,18114.67,5689.40,9425.13,3000.14,20.152123,13.705358,12.314239,5.5
3,Baja California Sur,6770.02,2853.31,3393.81,522.90,32.125232,50.445170,-21.310543,6.6
5,Campeche,3583.71,998.20,2389.27,196.24,-31.339774,-4.268257,-6.461134,5.2
6,Chiapas,3296.86,1644.83,1257.89,394.14,-13.311966,-8.555773,-9.842309,5.4
11,Chihuahua,40649.46,13138.94,15826.16,11684.36,13.231675,7.826500,8.319458,5.9
14,Ciudad de México,57968.76,25371.71,29826.01,2771.04,15.006572,13.451496,-1.547090,NaN
15,Colima,7672.50,3017.00,3179.64,1475.86,10.455324,7.511457,12.961593,4.9
16,Durango,8733.43,2462.20,2931.02,3340.21,-1.619555,8.706767,3.604147,5.7
17,Guanajuato,20738.33,9259.01,10634.28,845.04,18.593784,4.870970,-2.036987,6.6


In [27]:
#Merge Data
merge_df = pd.merge(clean_df_gp, clean_state, how = "inner", on = "Estado")
group_df = merge_df.groupby("Estado").sum()

group_df.drop(columns = "year")

,Capacidad de la central eléctrica (kW),Capacidad_kW_2017,Capacidad_kW_2018,Capacidad_kW_2019,Crecimiento_2017,Crecimiento_2018,Crecimiento_2019
Estado,,,,,,,
Aguascalientes,12519.23,5322.08,3963.01,3234.14,12.296030,18.638361,-0.158587
Baja California,18114.67,5689.40,9425.13,3000.14,20.152123,13.705358,12.314239
Baja California Sur,6770.02,2853.31,3393.81,522.90,32.125232,50.445170,-21.310543
Campeche,3583.71,998.20,2389.27,196.24,-31.339774,-4.268257,-6.461134
Chiapas,3296.86,1644.83,1257.89,394.14,-13.311966,-8.555773,-9.842309
Chihuahua,40649.46,13138.94,15826.16,11684.36,13.231675,7.826500,8.319458
Ciudad de México,57968.76,25371.71,29826.01,2771.04,15.006572,13.451496,-1.547090
Colima,7672.50,3017.00,3179.64,1475.86,10.455324,7.511457,12.961593
Durango,8733.43,2462.20,2931.02,3340.21,-1.619555,8.706767,3.604147


In [ ]:
group_df.insert(2, "Radiation", [5.6, , 24, 21], True) 
  
# Observe the result 
df 